This notebook follows the tutorial in the TINC python documentation: 
[TINC Python Tutorial](https://tinc-python.readthedocs.io/en/latest/tutorial.html)


In [ ]:
%pylab inline

In [ ]:
from tinc import *

In [ ]:
l = Parameter("length", "dimensions")
w = Parameter("width", "dimensions")
l.values = linspace(0, 10, 11)
w.values = linspace(0, 10, 11)

pos = Parameter("position")
pos.values = linspace(0, 1, 21)

In [ ]:
ps = ParameterSpace()
ps.register_parameters([l,w,pos])

In [ ]:
ps.set_root_path("/data/")
ps.set_current_path_template("w%%width%%_l%%length%%")

In [ ]:
ps.get_current_relative_path()

In [ ]:
l.ids = [str(i) for i in range(11)]
w.ids = [str(i) for i in range(11)]

In [ ]:
l.value = 4
w.value = 3
ps.set_current_path_template("w%%width:ID%%_l%%length:ID%%")
ps.get_current_relative_path()

In [ ]:
ps.get_parameter("width")

## Presets

In [ ]:
presets = PresetHandler()
presets.register_parameter(l)
presets.register_parameter(w)

In [ ]:
l.value = 2
w.value = 1
presets.store_preset("test1")

In [ ]:
l.value = 5
w.value = 3
presets.recall_preset("test1")
print(l.value, w.value)

In [ ]:
!cat presets/test1.preset

## Parameter callbacks

In [ ]:
def max_circle(value = 0):
    m = min(l.value, w.value)
    print(m)
    return m

In [ ]:
l.value = 4
w.value = 7
max_circle()

In [ ]:
l.register_callback(max_circle)
w.register_callback(max_circle)

In [ ]:
w.value = 5
l.interactive_widget()

## Parameter Sweeps

In [ ]:
import matplotlib.pyplot as plt
def graph(length, width, pos):    
    plt.axes()
    
    # Draw rectangle
    points = [[0, 0], [length, 0], [length, width], [0, width], [0,0]]
    polygon = plt.Polygon(points)
    plt.gca().add_patch(polygon)

    # Draw circle
    diameter = min(length, width)
    
    center = (diameter* 0.5 + (pos *(length - diameter)), diameter* 0.5 + (pos *(width - diameter)))
    
    circle = plt.Circle(center, radius=diameter* 0.5, fc='y')
    plt.gca().add_patch(circle)
    plt.axis('scaled')
    plt.xlim(0, 10)
    plt.ylim(0, 10)

In [ ]:
graph(4, 7, 0.5)

In [ ]:
graph(6,2, 0.8)

In [ ]:
def save_graph():
    graph(l.value, w.value, pos.value)

    filename = f"graph_{l.value}_{w.value}_{pos.value}.png"
    print("saving " + filename)
    savefig(filename)
    plt.close()

In [ ]:
save_graph()

In [ ]:
ps.sweep(save_graph, force_values=True)

In [ ]:
l.remove_callback(max_circle)
w.remove_callback(max_circle)

In [ ]:
ps.stop_sweep()

In [ ]:
ps.sweep(save_graph, force_values=True)

In [ ]:
ps.stop_sweep()

In [ ]:
def save_graph2(length, width, position):
    graph(length, width, position)

    filename = f"graph_{length}_{width}_{position}.png"
    print("saving " + filename)
    savefig(filename)

In [ ]:
ps.sweep(save_graph2)

In [ ]:
ps.stop_sweep()

## Cache


In [ ]:
ps.enable_cache()

In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import time
def create_graph(length, width, position):
    graph(length, width, position)
    f = gcf()
#     f.set_figwidth(5)
#     f.set_figheight(5)
    canvas = FigureCanvas(f)
    canvas.draw()
    image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    plt.close()
    
    print(f"{length}_{width}_{position}")
    time.sleep(2) # Fake a time consuming process
    return image.reshape((int(f.get_figheight()*f.get_dpi()), int(f.get_figwidth()*f.get_dpi()), 3))

In [ ]:
im = create_graph(1,3,0.3)

In [ ]:
import IPython.display
import PIL.Image
IPython.display.display(PIL.Image.fromarray(im))

In [ ]:
ps.stop_sweep()
ps.sweep(create_graph)

In [ ]:
%%time
im = ps.run_process(create_graph)

In [ ]:
%%time
im = ps.run_process(create_graph)

In [ ]:
ps.clear_cache()